In [11]:
import pandas as pd
import numpy as np
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import os
import json
from pathlib import Path
import pprint
import time

In [12]:
from dotenv import load_dotenv
load_dotenv()

groq_key = os.getenv("groq_key_paid")

data_path = os.getenv("data_path")
data_path = Path(data_path)

In [13]:
from main import main
from query import DataLoader

In [14]:
data_loader = DataLoader(data_path)

In [15]:
llm_name = "llama-3.1-70b-versatile" # "llama-3.3-70b-versatile" # "llama-3.1-70b-versatile"

context_column = "llm_summary"

In [16]:
experiment_name = f"exp1_llm-{llm_name}_context-{context_column}"
experiment_name

'exp1_llm-llama-3.1-70b-versatile_context-llm_summary'

In [17]:
def load_or_create_csv(experiment_name, df):
    try:
        # Attempt to read the CSV file
        curr_experiment = pd.read_csv(data_path / "Experiments" / f"{experiment_name}.csv")
        return curr_experiment
    except FileNotFoundError:
        # If the file does not exist, create it from the incoming DataFrame
        df.to_csv(data_path / "Experiments" / f"{experiment_name}.csv", index=False)
        return df

In [18]:
labeled_df = data_loader.get_label_df()

In [19]:
labeled_df = load_or_create_csv(experiment_name, labeled_df)

In [20]:
# Check if 'token_usage' column exists, if not create it
if 'token_usage' not in labeled_df.columns:
    labeled_df['token_usage'] = 0  # Initialize with zeros or any default value


for index, row in labeled_df.iterrows():
    if row["token_usage"] > 0:
        print('skip:',index)
        continue
    else:
        encounter_key = row["key"]
        print("label_index:",index," key:",encounter_key)
        (
            ret_recommendation,
            ret_reasoning,
            token_usage,
            search_history_so_far,
            token_count_history,
        ) = main(
            groq_key=groq_key, 
            data_path=data_path, 
            encounter_key=encounter_key,
            llm_name=llm_name,
            context_column=context_column,
        )
        
        print(ret_recommendation)
        labeled_df.at[index, 'ret_recommendation'] = ret_recommendation
        labeled_df.at[index, 'ret_reasoning'] = ret_reasoning
        labeled_df.at[index, 'token_usage'] = token_usage
        labeled_df.at[index, 'search_history_so_far'] = str(search_history_so_far)
        labeled_df.at[index, 'token_count_history'] = str(token_count_history)
        
        labeled_df.to_csv(data_path / "Experiments" / f'{experiment_name}.csv', index=False)
        
        # # # # # # # # # # # # #
        time.sleep(60) # s
        # # # # # # # # # # # # #

label_index: 0  key: D6253A5CE371EA
continue
label_index: 1  key: DFAFED1811B871
continue
label_index: 2  key: D4865B8BBB294E
deprescribe
label_index: 3  key: D00F3A8D5F43B2
deprescribe
label_index: 4  key: DA15CCF42ACF8B
continue
label_index: 5  key: DFB07E6B8F0957
deprescribe
label_index: 6  key: D1BD3665C06499
deprescribe
label_index: 7  key: D068E26FFF9F43
deprescribe
label_index: 8  key: D0E44FD0BBD96F
deprescribe
label_index: 9  key: D17AFAD1D7BB68
deprescribe
label_index: 10  key: DAA6DE9D2EC973
stop
label_index: 11  key: D5A7045ED60A2E
deprescribe
label_index: 12  key: D4BAEF91CA7588
deprescribe
label_index: 13  key: DACBDED2F35C67
deprescribe
label_index: 14  key: DA44DBBF3CCE49
deprescribe
label_index: 15  key: DAABF28BCE02C7
continue
label_index: 16  key: DB1D9E08BBEC87
continue
label_index: 17  key: D6F69E18CCF3AE
deprescribe
label_index: 18  key: D35D002F7350C0
deprescribe
label_index: 19  key: DF655634638393
deprescribe
label_index: 20  key: D8204A1E91DE84
deprescribe
lab

In [ ]:
labeled_df

In [23]:
text = str("""Based on the provided context, I will determine if there is evidence of the specific conditions to help decide whether to continue, reduce, or stop the PPI.\n\nHere is the output in the required JSON format:\n\n```\n{\n  "diagnosis_boolean": "1",\n  "explanation": "There is evidence of a history of gastrointestinal bleeding, as the patient experienced a lower GI bleed following a colonoscopy with polyp removal. Although the note does not definitively confirm severe esophagitis, bleeding esophagitis, or peptic ulcer disease, it mentions a GI bleed related to polyp removal, which suggests a history of gastrointestinal bleeding."\n}\n```\n\nHowever, to fully address the question, I must consider each condition individually:\n\n1. **Barretts Esophagus or esophageal cell changes**: There is no definitive evidence of Barretts Esophagus, but the upper endoscopy report mentions "mild esophageal changes" and "scattered 2-4 mm globules of white exudate," which may indicate some esophageal inflammation or changes.\n\n2. **Chronic Non-Steroidal Anti-Inflammatory (NSAID) use or GI prophylaxis NSAID use**: There is no mention of NSAID use in the provided context.\n\n3. **Severe esophagitis including bleeding esophagitis or esophageal ulcer**: The note does not definitively diagnose severe esophagitis, bleeding esophagitis, or esophageal ulcer.\n\n4. **History of gastrointestinal bleeding, gastric ulcer, upper GI bleed, or peptic ulcer hemorrhage**: There is evidence of a lower GI bleed following a colonoscopy with polyp removal, which suggests a history of gastrointestinal bleeding.\n\nGiven the information, the most relevant condition to the decision of continuing, reducing, or stopping the PPI is the history of gastrointestinal bleeding. The presence of mild esophageal changes and potential reflux symptoms also supports the use of a PPI but is not as directly relevant to the conditions listed in the question. \n\nTherefore, focusing on the condition with the most direct evidence:\n\n```\n{\n  "diagnosis_boolean": "1",\n  "explanation": "There is evidence of a history of gastrointestinal bleeding due to a lower GI bleed following a colonoscopy with polyp removal."\n}\n""")
text

'Based on the provided context, I will determine if there is evidence of the specific conditions to help decide whether to continue, reduce, or stop the PPI.\n\nHere is the output in the required JSON format:\n\n```\n{\n  "diagnosis_boolean": "1",\n  "explanation": "There is evidence of a history of gastrointestinal bleeding, as the patient experienced a lower GI bleed following a colonoscopy with polyp removal. Although the note does not definitively confirm severe esophagitis, bleeding esophagitis, or peptic ulcer disease, it mentions a GI bleed related to polyp removal, which suggests a history of gastrointestinal bleeding."\n}\n```\n\nHowever, to fully address the question, I must consider each condition individually:\n\n1. **Barretts Esophagus or esophageal cell changes**: There is no definitive evidence of Barretts Esophagus, but the upper endoscopy report mentions "mild esophageal changes" and "scattered 2-4 mm globules of white exudate," which may indicate some esophageal infla

In [24]:
# A simpler regex to match JSON-like structures
json_pattern = r'\{[\s\S]*?\}'

# Find all matches in the text
matches = re.findall(json_pattern, text, re.DOTALL)

In [25]:
for match in matches:
    print(match)

{
  "diagnosis_boolean": "1",
  "explanation": "There is evidence of a history of gastrointestinal bleeding, as the patient experienced a lower GI bleed following a colonoscopy with polyp removal. Although the note does not definitively confirm severe esophagitis, bleeding esophagitis, or peptic ulcer disease, it mentions a GI bleed related to polyp removal, which suggests a history of gastrointestinal bleeding."
}
{
  "diagnosis_boolean": "1",
  "explanation": "There is evidence of a history of gastrointestinal bleeding due to a lower GI bleed following a colonoscopy with polyp removal."
}


In [26]:
print(matches[-1])

{
  "diagnosis_boolean": "1",
  "explanation": "There is evidence of a history of gastrointestinal bleeding due to a lower GI bleed following a colonoscopy with polyp removal."
}


In [17]:
import re
match = re.search(json_pattern, text, re.DOTALL)
match

<re.Match object; span=(211, 2163), match='{\n  "diagnosis_boolean": "1",\n  "explanation": >

In [20]:
match.group()

'{\n  "diagnosis_boolean": "1",\n  "explanation": "There is evidence of a history of gastrointestinal bleeding, as the patient experienced a lower GI bleed following a colonoscopy with polyp removal. Although the note does not definitively confirm severe esophagitis, bleeding esophagitis, or peptic ulcer disease, it mentions a GI bleed related to polyp removal, which suggests a history of gastrointestinal bleeding."\n}\n```\n\nHowever, to fully address the question, I must consider each condition individually:\n\n1. **Barretts Esophagus or esophageal cell changes**: There is no definitive evidence of Barretts Esophagus, but the upper endoscopy report mentions "mild esophageal changes" and "scattered 2-4 mm globules of white exudate," which may indicate some esophageal inflammation or changes.\n\n2. **Chronic Non-Steroidal Anti-Inflammatory (NSAID) use or GI prophylaxis NSAID use**: There is no mention of NSAID use in the provided context.\n\n3. **Severe esophagitis including bleeding e

In [ ]:
encounter_key = "SyntheticPt5"

In [ ]:
(
    final_recommendation,
    final_explanation,
    token_usage,
    search_history_thus_far_list,
    token_count_history,
) = main(groq_key=groq_key, data_path=data_path, encounter_key=encounter_key)

In [ ]:
final_recommendation

In [ ]:
pprint.pprint(final_explanation)

In [ ]:
token_usage

In [ ]:
pprint.pprint(search_history_thus_far_list)

In [ ]:
pprint.pprint(token_count_history)